# Client

In [1]:
import cv2
import socket
import threading
import numpy 

In [2]:
# tcp
s = socket.socket(socket.AF_INET,socket.SOCK_STREAM)
host_ip = '169.254.208.239' # paste IP address here
port = 5000
s.connect((host_ip,port)) 
s.send(b"connection establised")
 
# accessing phone app : url of IP Webcam Android App 
cap = cv2.VideoCapture(1)

def recv_data():
    print("Entering to video mode")
    while True:
        data = s.recv(100000000)
        if (data == b'stop'):
            print("stopped")
            #camera release
            cap.release()
            # session closed
            s.close()
        else:
            #frombuffer -> Interpret a buffer as a 1-dimensional array.
            photo1 = numpy.frombuffer(data, dtype=numpy.uint8)
           
            if len(photo1) == 600*400*3: # changing the array shape and getting the video
                cv2.imshow('SERVER', photo1.reshape(400, 600, 3))
                if cv2.waitKey(1) == 13:
                    s.send(b'stop')
                    cap.release()
                    cv2.destroyAllWindows()
                    break
def sending_data():
    while True:
        ret, photo = cap.read()
        photo = cv2.resize(photo, (600,400))
        
        if ret:
            # converting array to bytes
            s.send(numpy.ndarray.tobytes(photo))
        else:
            print("not connected")
    cap.release()
            
a1 = threading.Thread(target=recv_data)
a2 = threading.Thread(target=sending_data)
a1.start()
a2.start()

Entering to video mode
